Modern Portfolio Theory
Modern portfolio theory also popularly called Mean-Variance Portfolio Theory (MVP) is a major breakthrough in finance. It is based on the premise that returns are normally distributed and by looking at mean and variance, we can essentially describe the distribution of end-of-period wealth.

The basic idea of this theory is to achieve diversification by constructing a portfolio for a minimal portfolio risk or maximal portfolio returns. Accordingly, the Efficient Frontier is a set of optimal portfolios in the risk-return spectrum, and portfolios located under the Efficient Frontier curve are considered sub-optimal.

This means that the portfolios on the frontier offered

Highest expected return for a given level of risk

Lowest level of risk for a given level of expected returns


In essence, the investors' goal should be to select a level of risk that he/she is comfortable with and then find a portfolio that maximizes returns based on the selected risk level.

In [126]:
import pandas as pd
import yfinance as yf
import numpy as np
from numpy.linalg import multi_dot
import cufflinks as cf
import plotly.express as px
import datetime

# Set the offline mode for cufflinks
cf.set_config_file(offline=True, dimensions=(1000, 600))
px.defaults.width, px.defaults.height = 1000, 600

# Set the random seed to ensure reproducibility
np.random.seed(0)

Retrive Data
We will retrieve price data from a list of stocks using the yfinance library. We will use the adjusted closing price for our analysis.


In [127]:
assets = ['AAPL', 'MSFT', 'AMZN', 'GOOG']

# Number of assets
num_assets = len(assets)

# Number of simulations for the optimisation process
num_simulations = 10000

# Start date
end_date = datetime.datetime(2018, 12, 31)
start_date = end_date - datetime.timedelta(days=365)

# Download the data
df = yf.download(assets, start=start_date, end=end_date)['Adj Close']
df.head()

[*********************100%***********************]  4 of 4 completed


,AAPL,AMZN,GOOG,MSFT
Date,,,,
2018-01-02,40.831589,59.450500,53.250000,80.562042
2018-01-03,40.824482,60.209999,54.124001,80.936974
2018-01-04,41.014103,60.479500,54.320000,81.649345
2018-01-05,41.481068,61.457001,55.111500,82.661636
2018-01-08,41.326996,62.343498,55.347000,82.746002


In [128]:
# Plot price history for 
df.iplot(kind='line')

In [129]:
# Dataframe of returns and volatility
returns = df.pct_change().dropna()
annual_returns = round(returns.mean()*260*100,2)
annual_volatility = round(returns.std()*np.sqrt(260)*100,2)

df1 = pd.DataFrame({
'Annualised Return': annual_returns,
'Annualised Volatility': annual_volatility
})

df1.head()

,Annualised Return,Annualised Volatility
AAPL,-4.38,29.24
AMZN,29.47,36.75
GOOG,1.33,28.64
MSFT,22.14,28.80


In [130]:
# Plot annualized return and volatility
df1.iplot(kind='bar', shared_xaxes=True, orientation='h', title='Annualized Return and Volatility')

### Portfolio Performance
Let's first assume an equal weights of assets in our portfolio and define some functions to analyse the performance of our portfolio.

In [133]:
# Create a list of equal weights
inital_weights = np.array([1/num_assets]*num_assets)

In [ ]:
# Create a function to calculate portfolio return, volatility and Sharpe ratio
def portfolio_Performance(weights):
    weights = np.array(weights)
    portfolio_return = np.sum(returns.mean()*weights)*260
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov()*260, weights)))
    portfolio_sharpe = portfolio_return/portfolio_volatility
    return np.array([portfolio_return, portfolio_volatility, portfolio_sharpe])

### Portfolio Statistics
Consider a portfolio which is fully invested in risky assets. Let $w$ and $\mu$ be the vector of weights and mean returns of $n$ assets.

$$
w=\left(\begin{array}{c}
w_{1} \\
w_{2} \\
\vdots \\
w_{n}
\end{array}\right) ; \mu=\left(\begin{array}{c}
\mu_{1} \\
\mu_{2} \\
\vdots \\
\mu_{n}
\end{array}\right)
$$

where the $\sum_{i=1}^{n} w_{i}=1$

Expected Portfolio Return is then the dot product of the expected returns and their weights.

$$
\mu_{\pi}=w^{T} \cdot \mu
$$

which is also equivalent to the $\sum_{i=1}^{n} w_{i} \mu_{i}$

Expected Portfolio Variance is then the multidot product of weights and the covariance matrix.

$$
\sigma_{\pi}^{2}=w^{T} \cdot \Sigma \cdot w
$$

where, $\Sigma$ is the covariance matrix

$$
\Sigma=\left(\begin{array}{ccc}
\Sigma_{1,1} & \ldots & \Sigma_{1, n} \\
\vdots & \ddots & \vdots \\
\Sigma_{n, 1} & \ldots & \Sigma_{n, n}
\end{array}\right)
$$

### Portfolio Simulation
Now, we will implement a Monte Carlo simulation to generate random portfolio weights on a larger scale and calculate the expected portfolio return, variance and sharpe ratio for every simulated allocation. We will then identify the portfolio with a highest return for per unit of risk.

In [120]:
def portfolio_Simulation(returns):

    # Calculate the mean returns and covariance matrix
    mean_returns = returns.mean()
    cov_matrix = returns.cov()

    # Simulate 10,000 portfolios
    weights = np.random((num_simulations, len(mean_returns)))
    weights /= np.sum(weights, axis=1)[:, np.newaxis]

    # Calculate the portfolio returns and volatility
    portfolio_returns = np.dot(weights, mean_returns) * 260
    portfolio_volatility = np.sqrt(np.sum((np.dot(weights, cov_matrix * 260) * weights), axis=1))

    # Create a DataFrame for analysis
    df = pd.DataFrame({
        'Portfolio Return': portfolio_returns,
        'Portfolio Volatility': portfolio_volatility
    })

    for i, symbol in enumerate(returns.columns):
        df[symbol + ' Weight'] = weights[:, i]

    df['Sharpe Ratio'] = df['Portfolio Return'] / df['Portfolio Volatility']

    return df

### Maximum Sharpe Ratio Portfolio

In [122]:
portfolio_simulation_df = portfolio_Simulation(returns)
portfolio_simulation_df.head()

,Portfolio Return,Portfolio Volatility,AAPL Weight,AMZN Weight,GOOG Weight,MSFT Weight,Sharpe Ratio
0,0.057882,0.285586,0.660578,0.279789,0.042493,0.017140,0.202678
1,0.131585,0.279335,0.265587,0.298076,0.198107,0.238230,0.471066
2,0.114570,0.289458,0.236638,0.407033,0.355137,0.001192,0.395810
3,0.011653,0.263867,0.477317,0.018697,0.406199,0.097787,0.044163
4,0.103113,0.275979,0.405812,0.254561,0.140906,0.198721,0.373627


### Visualise Monte Carlo Portfolio Simulation

### Efficient Frontier
The Efficient Frontier is a set of optimal portfolios in the risk-return spectrum, and portfolios located under the Efficient Frontier curve are considered sub-optimal.

Return objective:

$$
\operatorname{minimize}_{w_{1}, w_{2}, \ldots, w_{n}}^{2}\left(w_{1}, w_{2}, \ldots, w_{n}\right)
$$

subject to,

$$
E\left[R_{p}\right]=m
$$

Risk constraint:

$$
\underset{w_{1}, w_{2}, \ldots, w_{n}}{\operatorname{maximize}} E\left[R_{p}\left(w_{1}, w_{2}, \ldots, w_{n}\right)\right]
$$

subject to,

$$
\sigma_{p}^{2}\left(w_{1}, w_{2}, \ldots, w_{n}\right)=v^{2}
$$

where, $\sum_{i=1}^{n} w_{i}=1$ for the above objectives.

We can use numerical optimisation to achieve this objective. The goal is to find the optimal value of the objective function by adjusting the target variables operating within some boundary conditions and constraints.

### Constrained Optimization
Construction of optimal portfolios is a constrained optimisation problem where we specify some boundary conditions and constraints. The objective function here is a function returning maximum sharpe ratio, minimum variance (volatility) and the target variables are portfolio weights. We will use the minimize function from scipy optimization module to achieve our objective.